In [ ]:
from pyopensky.rest import REST

In [8]:
rest = REST()
rest.states()

,icao24,callsign,origin_country,last_position,timestamp,longitude,latitude,altitude,onground,groundspeed,track,vertical_rate,sensors,geoaltitude,squawk,spi,position_source
0,408127,SRR6565,United Kingdom,2025-11-25 23:29:56+00:00,2025-11-25 23:29:56+00:00,6.733,50.7331,3489.96,False,171.98,72.59,-9.43,None,3429.0,2005,False,0
1,a798d1,N589RC,United States,2025-11-25 23:29:56+00:00,2025-11-25 23:29:56+00:00,-123.2545,44.6154,2331.72,False,93.35,15.34,0.33,None,2346.96,<NA>,False,0
2,e8027b,LAN476,Chile,2025-11-25 23:29:57+00:00,2025-11-25 23:29:57+00:00,-60.4477,-34.6024,11727.18,False,235.22,278.68,8.78,None,12161.52,0403,False,0
3,a21b70,N235DT,United States,2025-11-25 23:29:56+00:00,2025-11-25 23:29:56+00:00,-115.137,35.9669,<NA>,True,7.72,180.0,<NA>,None,<NA>,<NA>,False,0
4,ad4f1d,N957ES,United States,2025-11-25 23:29:56+00:00,2025-11-25 23:29:56+00:00,-97.9677,35.7013,998.22,False,66.5,145.08,0.0,None,1036.32,<NA>,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,a73ed2,N566SP,United States,2025-11-25 23:29:54+00:00,2025-11-25 23:29:55+00:00,-97.0481,32.6017,579.12,False,49.4,1.19,0.33,None,579.12,<NA>,False,0
9154,a9c380,RPA3676,United States,2025-11-25 23:29:57+00:00,2025-11-25 23:29:57+00:00,-84.5391,38.952,2316.48,False,149.62,355.66,-6.5,None,2293.62,7077,False,0
9155,a05dba,UAL279,United States,2025-11-25 23:29:56+00:00,2025-11-25 23:29:56+00:00,-82.5406,27.9736,<NA>,True,3.86,115.31,<NA>,None,<NA>,2543,False,0
9156,a30296,N293WC,United States,2025-11-25 23:29:56+00:00,2025-11-25 23:29:56+00:00,-108.6646,37.5095,3261.36,False,64.36,145.42,-0.65,None,3337.56,<NA>,False,0
